In [ ]:
# import all the modules
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl.preprocess import FAST
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.freesurfer import FSCommand, MRIConvert
from pandas import DataFrame, Series, read_csv

# FSL setup- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTIGZ')

#freesurfer setup
fs_dir = '/home/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# Study specific variables
analysis_home = '/home/camachocm2/Analysis/Aggregate_anats'

workflow_dir = analysis_home + '/workflows'
proc_dir = analysis_home + '/proc/subj_data'
group_dir = analysis_home + '/proc/group_data'

#subject_info = read_csv(analysis_home + '/misc/subjs.csv', index_col=0)
#template_subjects = subject_info[subject_info['fsID'] where subject_info['templateID']==1].tolist()
#all_subjects = subject_info['fsID'].tolist()
template_subjects = []

In [ ]:
##### Set up nodes for file handling #####

### First, template creation ###
template_source = MapNode(FreeSurferSource(subjects_dir = fs_subjdir), 
                          name = 'template_source', 
                          iterfield = ['subject_id'])
template_source.inputs.subject_id = template_subjects

#set up datasink
datasink = Node(DataSink(base_directory = template_proc),
                name = 'datasink')

In [ ]:
######### Template creation functions #########
def make3DTemplate(subject_T1s, num_proc, output_prefix):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from os.path import abspath, split
    from os import getcwd
    from shutil import copyfile
    from glob import glob
    from subprocess import call

    curr_dir = getcwd()

    #copy T1s into current directory
    for T in range(0,len(subject_T1s)):
        [dirname,filename] = split(subject_T1s[T])
        copyfile(subject_T1s[T],curr_dir + '/S' + str(T)+'_'+filename)

    # -c flag is control for local computing (2= use localhost; required for -j flag)
    # -j flag is for number of processors allowed
    call(['antsMultivariateTemplateConstruction2.sh', '–d','3','–o', output_prefix,'–r','1','–c','2','–j', str(num_proc), '*.nii.gz'])
    
    sample_template = abspath(output_prefix + 'template0.nii.gz')
    
    return(sample_template)


In [ ]:
######### Template creation nodes #########

#convert freesurfer brainmask files to .nii
convertT1 = MapNode(MRIConvert(out_file='T1.nii.gz',
                               out_type='niigz'), 
                    name='convertT1', 
                    iterfield = ['in_file'])

#reorient files to standard space
reorientT1 = MapNode(Reorient2Std(),
                     name = 'reorientT1',
                     iterfield = ['in_file'])

#pass files into template function (normalized, pre-skull-stripping)
makeTemplate = Node(Function(input_names=['subject_T1s','num_proc','output_prefix'],
                             output_names=['sample_template'],
                             function=make3DTemplate),
                    name='makeTemplate')
makeTemplate.inputs.num_proc=4 
makeTemplate.inputs.output_prefix='Child_'

In [ ]:
######### Template creation workflow #########
template_flow = Workflow(name = "template_flow")
template_flow.connect([(fs_source, convertT1, [('T1','in_file')]),
                       (convertT1, reorientT1, [('out_file', 'in_file')]),
                       (reorientT1, makeTemplate, [('out_file', 'subject_T1s')]),
                       (reorientT1, datasink, [('out_file','proc_T1s')]),
                       (makeTemplate, datasink, [('sample_template', 'sample_template')])
                      ])

template_flow.base_dir = workflow_dir
template_flow.write_graph(graph2use = 'flat')
template_flow.run(plugin='Multiproc',plugin_args={'memory_gb':20})

In [ ]:
############ Tissue map functions ###############
def split_gm_from_aseg(aseg):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    
    
    return(masks)

In [ ]:
############ Tissue map nodes ###############
aseg_to_nii = Node(MRIConvert(out_file='aseg.nii.gz',
                              out_type='niigz'), 
                   name='aseg_to_nii')

reorient_aseg = Node(Reorient2Std(), 
                     name='reorient_aseg')


segment_brainmask = Node(FAST(), 
                         name='segment_brainmask')